In [ ]:
%pip install librosa
%pip install tensorflow
%pip install pandas
%pip install matplotlib
%pip install tqdm

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import librosa as lb
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
metadata = pd.read_csv('notes_v2/n_metadata.csv')
print(f'Samples num: {metadata.shape[0]}\nColumns num: {metadata.shape[1]}')
metadata.head()

In [ ]:
classes = metadata.groupby('Class_ID')['Class'].unique()
classes

# Feature Extraction Function

In [ ]:
def feature_extractor(path: str)  -> np.array:
    data, _ = lb.load(path)
    data = lb.feature.mfcc(data, n_mfcc=128)
    data = np.mean(data, axis=1)

    return data

# Data Extraction

In [ ]:
x, y = [], []

for i, rows in tqdm(metadata.iterrows()):
    path = rows['Sample']
    x.append(feature_extractor(f'../input/guitar-notes-sound{path[7:]}'))
    y.append(rows['Class_ID'])

x = np.array(x)
y = np.array(y)

x.shape, y.shape

In [ ]:
y = to_categorical(y)
y.shape

# Split data into training , validation and test

In [ ]:
xtrainval, xtest, ytrainval, ytest = train_test_split(x, y, test_size=0.1, stratify=y, random_state=42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrainval, ytrainval, test_size=0.2, stratify=ytrainval,random_state=42)

print('\nNumber of samples for Train set :',xtrain.shape[0])
print('Number of samples for Validation set :',xvalid.shape[0])
print('Number of samples for Test set :',xtest.shape[0])

# Model

In [ ]:
model = Sequential(
    [
        layers.Dense(1000, activation='relu', input_shape=(128,)),
        layers.Dense(750, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(250, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(14, activation='softmax')
    ]
)

model.summary()

# Training Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
training = model.fit(xtrain, ytrain, validation_data=(xvalid, yvalid), epochs=20)

# Verifying Results

In [ ]:
train_hist = pd.DataFrame(training.history)
train_hist

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(train_hist[['loss', 'val_loss']])
plt.legend(['loss', 'val_loss'])
plt.title('Loss per Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

plt.figure(figsize=(20, 8))
plt.plot(train_hist[['accuracy', 'val_accuracy']])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('Accuracy per Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
ytrue = np.argmax(ytest, axis=1)
ypred = np.argmax(model.predict(xtest), axis=1)
print('\nConfusion Matrix: \n\n')
print(confusion_matrix(ytrue, ypred))
print('\n\nClassification Report: \n\n', classification_report(ytrue, ypred))